In [101]:
import pandas as pd
import scipy
from scipy import stats
from scipy import linalg
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss
import pingouin as pg
from scipy.stats import levene
from statsmodels.stats.multicomp import (pairwise_tukeyhsd,
                                         MultiComparison)
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

import statsmodels.api as sm
import statsmodels.formula.api as smf

import scikit_posthocs as sp

In [61]:
df = pd.read_csv('/Users/siarheimasliankou/Documents/Work/DA_Practice/Statistics/Task_7/cars.csv')

In [62]:
df['company'] = df['CarName'].apply(lambda x: x.split()[0])

In [63]:
df = df.drop(columns=['CarName', 'car_ID'])

In [64]:
df.company.nunique()

28

In [65]:
df['company'] = df['company'].str.lower()
df['company'] = df['company'].replace({'maxda':'mazda', 'porcshce':'porsche', 'toyouta':'toyota', 'vokswagen':'volkswagen', 'vw':'volkswagen'})


In [66]:
df.company.value_counts()

company
toyota         32
nissan         18
mazda          17
mitsubishi     13
honda          13
volkswagen     12
subaru         12
peugeot        11
volvo          11
dodge           9
buick           8
bmw             8
audi            7
plymouth        7
saab            6
porsche         5
isuzu           4
jaguar          3
chevrolet       3
alfa-romero     3
renault         2
mercury         1
Name: count, dtype: int64

In [67]:
df

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,company
0,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero
1,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero
2,1,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-romero
3,2,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,2,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0,audi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,...,mpfi,3.78,3.15,9.5,114,5400,23,28,16845.0,volvo
201,-1,gas,turbo,four,sedan,rwd,front,109.1,188.8,68.8,...,mpfi,3.78,3.15,8.7,160,5300,19,25,19045.0,volvo
202,-1,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,...,mpfi,3.58,2.87,8.8,134,5500,18,23,21485.0,volvo
203,-1,diesel,turbo,four,sedan,rwd,front,109.1,188.8,68.9,...,idi,3.01,3.40,23.0,106,4800,26,27,22470.0,volvo


In [68]:
selected_columns = ['company', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'wheelbase', 'carlength', 'carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio', 'horsepower', 'price']
up_df = df[selected_columns]

up_df

,company,fueltype,aspiration,carbody,drivewheel,wheelbase,carlength,carwidth,curbweight,enginetype,cylindernumber,enginesize,boreratio,horsepower,price
0,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111,13495.0
1,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111,16500.0
2,alfa-romero,gas,std,hatchback,rwd,94.5,171.2,65.5,2823,ohcv,six,152,2.68,154,16500.0
3,audi,gas,std,sedan,fwd,99.8,176.6,66.2,2337,ohc,four,109,3.19,102,13950.0
4,audi,gas,std,sedan,4wd,99.4,176.6,66.4,2824,ohc,five,136,3.19,115,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,volvo,gas,std,sedan,rwd,109.1,188.8,68.9,2952,ohc,four,141,3.78,114,16845.0
201,volvo,gas,turbo,sedan,rwd,109.1,188.8,68.8,3049,ohc,four,141,3.78,160,19045.0
202,volvo,gas,std,sedan,rwd,109.1,188.8,68.9,3012,ohcv,six,173,3.58,134,21485.0
203,volvo,diesel,turbo,sedan,rwd,109.1,188.8,68.9,3217,ohc,six,145,3.01,106,22470.0


In [69]:
numeric_df = up_df[['wheelbase', 'carlength', 'carwidth', 'curbweight', 'enginesize', 'boreratio', 'horsepower', 'price']]

correlation_matrix = numeric_df.corr()

price_correlation = correlation_matrix['price']
price_correlation.round(2)

wheelbase     0.58
carlength     0.68
carwidth      0.76
curbweight    0.84
enginesize    0.87
boreratio     0.55
horsepower    0.81
price         1.00
Name: price, dtype: float64

In [70]:
correlation_matrix

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price
wheelbase,1.000000,0.874587,0.795144,0.776386,0.569329,0.488750,0.353294,0.577816
carlength,0.874587,1.000000,0.841118,0.877728,0.683360,0.606454,0.552623,0.682920
carwidth,0.795144,0.841118,1.000000,0.867032,0.735433,0.559150,0.640732,0.759325
curbweight,0.776386,0.877728,0.867032,1.000000,0.850594,0.648480,0.750739,0.835305
enginesize,0.569329,0.683360,0.735433,0.850594,1.000000,0.583774,0.809769,0.874145
boreratio,0.488750,0.606454,0.559150,0.648480,0.583774,1.000000,0.573677,0.553173
horsepower,0.353294,0.552623,0.640732,0.750739,0.809769,0.573677,1.000000,0.808139
price,0.577816,0.682920,0.759325,0.835305,0.874145,0.553173,0.808139,1.000000


In [71]:
df_dummy = pd.get_dummies(data=df[['fueltype', 'aspiration', 'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'enginetype', 'cylindernumber', 'fuelsystem', 'company']], drop_first = True)

In [72]:
df_dummy

,fueltype_gas,aspiration_turbo,doornumber_two,carbody_hardtop,carbody_hatchback,carbody_sedan,carbody_wagon,drivewheel_fwd,drivewheel_rwd,enginelocation_rear,...,company_nissan,company_peugeot,company_plymouth,company_porsche,company_renault,company_saab,company_subaru,company_toyota,company_volkswagen,company_volvo
0,True,False,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,True,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,True,False,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
201,True,True,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
202,True,False,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
203,False,True,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True


In [97]:
df_dummy_up = pd.get_dummies(data=up_df[['company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower', 'price']], drop_first = True)

df_dummy_up = df_dummy_up.astype(int)

df_dummy_up

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price,company_audi,company_bmw,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,88,168,64,2548,130,3,111,13495,0,0,...,0,0,0,0,0,1,0,0,0,0
1,88,168,64,2548,130,3,111,16500,0,0,...,0,0,0,0,0,1,0,0,0,0
2,94,171,65,2823,152,2,154,16500,0,0,...,0,0,1,0,0,0,1,0,0,0
3,99,176,66,2337,109,3,102,13950,1,0,...,1,0,0,0,0,1,0,0,0,0
4,99,176,66,2824,136,3,115,17450,1,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,109,188,68,2952,141,3,114,16845,0,0,...,1,0,0,0,0,1,0,0,0,0
201,109,188,68,3049,141,3,160,19045,0,0,...,1,0,0,0,0,1,0,0,0,0
202,109,188,68,3012,173,3,134,21485,0,0,...,0,0,1,0,0,0,1,0,0,0
203,109,188,68,3217,145,3,106,22470,0,0,...,1,0,0,0,0,0,1,0,0,0


In [75]:
# Предиктор (horsepower):
X = df_dummy_up['horsepower']

# Целевая переменная (price):
Y = df_dummy_up['price']

# Добавляем константу для учета пересечения (intercept) в модели:
X = sm.add_constant(X)

# Создаем и обучаем модель
model = sm.OLS(Y, X)
results = model.fit()

# Выводим результаты
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Sat, 27 Jul 2024   Prob (F-statistic):           1.48e-48
Time:                        11:36:44   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3721.7615    929.849     -4.003      0.0

In [98]:
# Предикторы:
Z = df_dummy_up.drop(['price'], axis = 'columns')

# Целевая переменная (price):
R = df_dummy_up['price']

# Добавляем константу для учета пересечения (intercept) в модели:
Z = sm.add_constant(Z)

# Создаем и обучаем модель
model = sm.OLS(R, Z)
results = model.fit()

# Выводим результаты
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.958
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     77.51
Date:                Sat, 27 Jul 2024   Prob (F-statistic):           1.41e-87
Time:                        12:51:27   Log-Likelihood:                -1808.6
No. Observations:                 205   AIC:                             3711.
Df Residuals:                     158   BIC:                             3867.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -3.226e+

In [99]:
#Выбираем колонки без марок машин:
cols_to_keep = df_dummy_up.columns[~ df_dummy_up.columns.str.startswith('company_')]
cols_to_keep

Index(['wheelbase', 'carlength', 'carwidth', 'curbweight', 'enginesize',
       'boreratio', 'horsepower', 'price', 'fueltype_gas', 'aspiration_turbo',
       'carbody_hardtop', 'carbody_hatchback', 'carbody_sedan',
       'carbody_wagon', 'drivewheel_fwd', 'drivewheel_rwd', 'enginetype_dohcv',
       'enginetype_l', 'enginetype_ohc', 'enginetype_ohcf', 'enginetype_ohcv',
       'enginetype_rotor', 'cylindernumber_five', 'cylindernumber_four',
       'cylindernumber_six', 'cylindernumber_three', 'cylindernumber_twelve',
       'cylindernumber_two'],
      dtype='object')

In [100]:
Z_2 = df_dummy_up[cols_to_keep].drop(['price'], axis = 'columns')

R_2 = df_dummy_up['price']

Z_2 = sm.add_constant(Z_2)

# Создаем и обучаем модель
model = sm.OLS(R_2, Z_2)
results = model.fit()

# Выводим результаты
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.46
Date:                Sat, 27 Jul 2024   Prob (F-statistic):           8.51e-81
Time:                        13:10:40   Log-Likelihood:                -1881.4
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -1.161e+

In [ ]:
#Выводы:

#Если судить чисто по диагностическим показателям (вроде R2), то модель со всеми предикторами лучшая.

#Большинство коэффициентов, связанных с марками машин, статистически незначимы.